In [1]:
from dotenv import load_dotenv
import sys
sys.path.append('..')


load_dotenv()

True

In [2]:
from src.vector import create_file_content_map
import os

file_content_map = create_file_content_map(root_dir=os.path.join("..", "src"), accept_patterns=["*.py"])


INFO:faiss.loader:Loading faiss with AVX512 support.
INFO:faiss.loader:Could not load library with AVX512 support due to:
ModuleNotFoundError("No module named 'faiss.swigfaiss_avx512'")
INFO:faiss.loader:Loading faiss with AVX2 support.
INFO:faiss.loader:Successfully loaded faiss with AVX2 support.


Found 17 valid files


In [3]:
from src.vector import EmbeddingManager, OpenAIEmbeddingProvider, FAISSVectorStore, EmbeddingConfig


embedding_provider = OpenAIEmbeddingProvider(config=EmbeddingConfig(model_name="text-embedding-3-small"))
vector_store = FAISSVectorStore(dimension=1536, index_path="faiss_index.bin")

embedding_manager = EmbeddingManager(
    embedding_provider=embedding_provider,
    vector_store=vector_store,
)

In [4]:
texts = list(file_content_map.values())
metadatas = [{"source": file_path} for file_path in file_content_map.keys()]

await embedding_manager.add_texts(texts=texts, metadatas=metadatas)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [7]:
results = await embedding_manager.similarity_search(query="What is the purpose of the EmbeddingManager class?", limit=10)
results = sorted(results, key=lambda x: x.score, reverse=True)

for result in results:
    print(result.score)
    print(result.text[:10])
    print("-" * 100)


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


1.8231627941131592
from .watc
----------------------------------------------------------------------------------------------------
1.8073091506958008
import log
----------------------------------------------------------------------------------------------------
1.720302939414978
from typin
----------------------------------------------------------------------------------------------------
1.5216126441955566
from datac
----------------------------------------------------------------------------------------------------
1.4744725227355957
import os

----------------------------------------------------------------------------------------------------
1.4427917003631592
import ric
----------------------------------------------------------------------------------------------------
1.1858773231506348
import asy
----------------------------------------------------------------------------------------------------
1.1801719665527344
from abc i
------------------------------------------------------

In [5]:
from src.query import QueryProcessor, QueryType
from langchain_openai import ChatOpenAI

query_processor = QueryProcessor(embedding_manager=embedding_manager, llm=ChatOpenAI(model="gpt-4o-mini"))

In [9]:
query_results = await query_processor.process_query(query="What is the purpose of the VectorStore class?", query_type=QueryType.SEMANTIC)

query_results.analysis


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


AIMessage(content="### Analysis of the VectorStore Class\n\n#### 1. How Each Section Relates to the Query\n\nThe main focus of the query is the `VectorStore` class, which is situated within the context of a broader codebase that deals with the management of embedding vectors. The class is defined in the file `vector.py`, and its purpose is to serve as an abstract base class for various types of vector stores. The code sections that involve `VectorStore` include:\n\n- The `VectorStore` abstract class itself, which defines essential methods for managing vectors.\n- Concrete implementations of `VectorStore` such as `ChromaVectorStore`, `FAISSVectorStore`, and `PGVectorStore`. Each of these classes provides specific functionalities for handling vectors in different storage systems.\n- The `EmbeddingManager` class, which utilizes the `VectorStore` to manage embeddings and interface with the vector store.\n\n#### 2. Key Implementation Details\n\n- **Abstract Class**: `VectorStore` is an abst

In [10]:
with open("result.md", "w") as f:
    f.write(query_results.analysis.content)



In [6]:
from src.query import StreamQueryProcessor
from langchain_openai import ChatOpenAI

stream_query_processor = StreamQueryProcessor(embedding_manager=embedding_manager, llm=ChatOpenAI(model="gpt-4o-mini"))

async for result in stream_query_processor.stream_query_result(query="What is the purpose of the VectorStore class?", query_type=QueryType.CODE_PATTERN):
    
    with open("result.md", "a") as f:
        f.write(result.text)




INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
